In [20]:
import pandas as pd
import numpy as np
import json
from SPARQLWrapper import SPARQLWrapper, JSON
%matplotlib inline

pd.set_option('display.max_rows', 100)

## Load Data

In [2]:
df_legs = pd.read_csv(r'..\data\interim\legs.csv', dtype = {'VORIHORAINI':str, 'VDESHORAFIN':str})
df_trips = pd.read_csv(r'..\data\interim\trips.csv', dtype = {'VORIHORAINI':str, 'VDESHORAFIN':str})
df_people = pd.read_csv(r'..\data\interim\people.csv', dtype = {'VORIHORAINI':str, 'VDESHORAFIN':str})

codes = pd.read_csv(r'..\data\processed\codes_translated.csv', dtype = {'CODE': float})

In [36]:
df_legs.head()

,ID_HOGAR,ID_IND,ID_VIAJE,CODMUNI,NOMMUNI,CODPROV,NOMPROV,ZT1259,CZ208,ELE_HOGAR_NUEVO,...,ESUBIDA_cod,EBAJADA,EBAJADA_cod,ETITULO,EESTACIONA,EOCUPACION,ETDESPH,TIPO_ENCUESTA_etap,COD_MUN_PARADA,ELE_G_POND_Esc2
0,189,1,1.0,2,AJALVIR,28,MADRID,002-002,60,69.902334,...,NaN,NaN,NaN,NaN,1.0,1.0,3.0,CAPI,NaN,66.304668
1,189,1,2.0,2,AJALVIR,28,MADRID,002-002,60,69.902334,...,NaN,NaN,NaN,NaN,4.0,1.0,3.0,CAPI,NaN,66.304668
2,189,2,NaN,2,AJALVIR,28,MADRID,002-002,60,69.902334,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,244,1,1.0,2,AJALVIR,28,MADRID,002-002,60,67.889119,...,NaN,NaN,NaN,NaN,1.0,1.0,3.0,CAPI,NaN,66.304668
4,244,1,2.0,2,AJALVIR,28,MADRID,002-002,60,67.889119,...,NaN,NaN,NaN,NaN,1.0,1.0,3.0,CAPI,NaN,66.304668


## Match Stations of ESUBIDA and EBAJADA columns

In [29]:
# Those are all stations
df_legs.ESUBIDA.unique()

array([nan, 'CTRA.DAGANZO-CTRA.COBEÑA', 'CANILLEJAS', ...,
       'PZA.GENERALISIMO-AYUNTAMIENTO', 'AV.MADRID-CTRA.CAMPO REAL',
       'AV.MADRID-Cº ESTRECHILLO'], dtype=object)

In [38]:
# Those are all station codes, but I can't find them online
df_legs.ESUBIDA_cod.unique()

array([nan, '8002256', '74', ..., '8035322', '8025351', '8025350A'],
      dtype=object)

Idea: get them from this list https://es.wikipedia.org/wiki/Anexo:Estaciones_del_Metro_de_Madrid
This can be done using its wikidata entry here: https://www.wikidata.org/wiki/Q6577514
It can be done using the following SPARQL query:

In [39]:
q = """
    SELECT ?item ?itemLabel ?coords
    {
      ?item wdt:P31/wdt:P279* wd:Q928830 ;   # this translates to "is a instace of" "metro station"
            wdt:P131 wd:Q2807 ;              # this translates to "located in the administrative territorial entity" "Madrid"
            wdt:P625 ?coords .
      SERVICE wikibase:label { bd:serviceParam wikibase:language "es" . }
    } 
"""

Test here: https://query.wikidata.org/#SELECT%20%3Fitem%20%3FitemLabel%20%3Fcoords%0A%7B%0A%20%20%3Fitem%20wdt%3AP31%2Fwdt%3AP279%2a%20wd%3AQ928830%20%3B%0A%20%20%20%20%20%20%20%20wdt%3AP131%20wd%3AQ2807%20%3B%0A%20%20%20%20%20%20%20%20wdt%3AP625%20%3Fcoords%20.%0A%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22es%22%20.%20%7D%0A%7D%20

In [40]:
# not working with current pandas version (see issue)
# !pip install sparql-dataframe

In [41]:
!pip install sparqlwrapper

In [42]:
def get_sparql_dataframe(service, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    From here: https://lawlesst.github.io/notebook/sparql-dataframe.html
    """
    sparql = SPARQLWrapper(service)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)
endpoint = "https://query.wikidata.org/sparql"


q = """
    SELECT ?item ?itemLabel ?coords
    {
      ?item wdt:P31/wdt:P279* wd:Q928830 ;
            wdt:P131 wd:Q2807 ;
            wdt:P625 ?coords .
      SERVICE wikibase:label { bd:serviceParam wikibase:language "es" . }
    } 
"""

In [43]:
df = get_sparql_dataframe(endpoint, q)
df

,item,itemLabel,coords
0,http://www.wikidata.org/entity/Q2244021,Estación de Bambú,Point(-3.676419444 40.476813888)
1,http://www.wikidata.org/entity/Q2244081,Estación de Estadio Metropolitano,Point(-3.6001502 40.4333937)
2,http://www.wikidata.org/entity/Q2244102,Estación de Las Tablas,Point(-3.66941 40.5088)
3,http://www.wikidata.org/entity/Q2305840,Estación de Acacias / Embajadores,Point(-3.70269 40.4051)
4,http://www.wikidata.org/entity/Q2378663,Estación de Ventura Rodríguez,Point(-3.7138 40.4272)
...,...,...,...
177,http://www.wikidata.org/entity/Q3941077,Estación de Ronda de la Comunicación,Point(-3.66274 40.5155)
178,http://www.wikidata.org/entity/Q5846080,Estación de Paco de Lucía,Point(-3.7097 40.4997)
179,http://www.wikidata.org/entity/Q19950924,Chamartín,Point(-3.68272 40.4721)
180,http://www.wikidata.org/entity/Q19951196,Q19951196,Point(-3.70801 40.4249)


In [44]:
for station in df_legs.ESUBIDA.unique():
    print(station)

nan
CTRA.DAGANZO-CTRA.COBEÑA
CANILLEJAS
SUANZES
AV.LOGROÑO-Pº ALAMEDA DE OSUNA
EL CARMEN
AV.AMERICA-CANILLEJAS
AV.FRONTERAS-MADRID
AJALVIR-CTRA.DAGANZO
REAL-ATARRE
MARTIRES-RESIDENCIA 3ª EDAD
SAN ROQUE-GALILEO
CURATO-ESCUELAS
HOSPITAL REY JUAN CARLOS
CUATRO VIENTOS
ALONSO MARTINEZ
GOYA
PRINCIPE PIO
PROSPERIDAD
MARTIRES-LOS POCILLOS
INTERCAMBIADOR DE PRINCIPE PIO
CTRA.CCHEL.A ARAVACA-COLONIA JARDIN
PZA.ESTACION-EST.POZUELO
COLONIA JARDIN
Pº EXTREMADURA-CUATRO VIENTOS
AV.MADRID-CAÑO VIEJO
Pº  EXTREMADURA-CARLINA
PZA.JACINTO BENAVENTE
Pº  EXTREMADURA-BATAN
AV.MADRID-TVRA.DEL BRONCE
LOS OLIVOS-CASARRUBIOS
AV.ANDALUCIA-VILLAVERDE BAJO CRUCE
VIA COMPLUTENSE-BRIHUEGA
AV.CONSTITUCION-CANTALARRANA
RONDA FISCAL-Pº LAS MORERAS
VIA COMPLUTENSE-RIBERA
Cº OLIVO-AV.LUZ
ALCALA DE HENARES
TORREJON DE ARDOZ
ALALPARDO-CLUB DE JUBILADOS
PZA.RODRIGUEZ MARIN-AYUNTAMIENTO
AV.AJALVIR-AMADEO VIVES
AVENIDA DE AMERICA
SAINZ DE BARANDA
INTERCAMBIADOR DE AVENIDA AMERICA
AV.DAGANZO-GTA.DEL CHORRILLO
AV.MADRID-PUERT

JOSEFA VALCARCEL-GUTIERREZ CANALES
GTA. MAR DE CRISTAL
NTRA.SRA.FATIMA-EUGENIA DE MONTIJO
Pº  MUÑOZ GRANDES-OCA
GOMEZ ARTECHE-AV.CARABANCHEL ALTO
VELAZQUEZ-VILLANUEVA
DIEGO DE LEON-CASTELLO
DR.ESQUERDO-AV.NAZARET
Pº  DE LA DIRECCION-JARAMAGOS
AV.DE AMERICA-I.E.S.BARAJAS
SAINZ DE BARANDA-MAIQUEZ
AV.MENENDEZ PELAYO-MENORCA
MENDEZ ALVARO-EST.SUR AUTOBUSES
TRVA.NAVALCARBON-HERON CITY
VALLE DE ORO-CNO.VIEJO DE LEGANES
PALOS DE LA FRONTERA-Pº  STA.Mª  CABEZA
AV.ORELLANA-PZA.DE LA INMACULADA
DR.ESQUERDO-HERMOSILLA
AVDA. CDAD.BARCELONA-DR.ESQUERDO
DR.ESQUERDO-CAVANILLES
DR.ESQUERDO-SAINZ DE BARANDA
AV.REINA VICTORIA-AV.PABLO IGLESIAS
AV.GREGORIO DEL AMO-M.BARTOLOME
DR.FEDERICO RUBIO-JERONIMA LLORENTE
AV.REINA VICTORIA-CUATRO CAMINOS
TAJO-UNIVERSIDAD
CONCHA ESPINA-ESTADIO SANTIAGO BERNABEU
PASEO CASTELLANA-RAFAEL SALGADO
PASEO CASTELLANA-ROSARIO PINO
Pº  CASTELLANA-JUZGADOS
GTA.EMBAJADORES
ALCOCER-VILLAVERDE CRUCE
AV.CARDENAL HERRERA ORIA-ALICIA BAENA
AV.COVIBAR-CENTRO DE SALUD
STA.ENGRACIA-BRE

AV.CONSUELO-DOCTOR RIVAS
SAN VICENTE DE PAUL-DOCTOR FLEMING
PARLA-VICENTE ALEIXANDRE
DOCTOR FLEMING-DOCTOR CASTELO
BADAJOZ-ETSI INFORMATICOS
FERROCARRIL-EST.PINTO
MANUEL DE FALLA-ESCUELA DE MUSICA
LIBERTAD-RONDA COMUNIDADES
SESEÑA-ILLESCAS
Pº ENRIQUE TIERNO GALVAN-COLEGIO
GTA.LAS AMAZONAS-HOMERO
Pº COMUNIDAD DE MADRID-CEMENTERIO
RDA.COMUNIDADES-RESIDENCIA
ROSALIA DE CASTRO-GTA.EULALIA ABAITUA
Pº ESTACION-IBIZA
AV.MAR MEDITERRANEO-AFRODITA
AV.MAR MEDITERRANEO-VENUS
AV.ANDALUCIA-Pº DE LA ESTACION
AV.MAR MEDITERRANEO-MINERVA
AGUADO-AV.HISPANOAMERICA
AV.ESPAÑA-JAMAICA
APOLO-JUPITER
AV.ANDALUCIA-POL.IND.ALVENA
AV.EUROPA-ESLOVAQUIA
AV.EUROPA-INGLATERRA
AV.SAN LUIS-GOLFO DE SALONICA
AV.EUROPA-BENELUX
VIRGILIO-ATLANTIDA
GTA.SIRENAS-AV.MAR MEDITERRANEO
GTA.JOSE Mª PEMAN-ANTONIO VAN DE PERE
DALI-PISCINA MUNICIPAL
AGUSTINA ARAGON-GTA.JOSEFINA CARABIAS
TRABAJADORAS COTTON-GTA.MARIANA PINEDA
AV.MAR EGEO-POSEIDON
ROSALIA DE CASTRO-LILI ALVAREZ
AV.ESPAÑA-AV.MAR EGEO
GTA.LAS SIRENAS-AV.MEDITERRANEO
AG

SAUCEDA-AV.JUAN AGÜI
CAÑADA REAL-COLEGIO
GOLONDRINA-VALTRAVIESO
FELIPE II-PADILLA
AV.ESPAÑA-CONSTITUCION
DAOIZ Y VELARDE-ESCUELA OFICIAL IDIOMAS
CTRA.M100-POL.IND.LA PEÑA
AV.CARABANCHEL ALTO-AV DEL EURO
SILVANO-HAENDEL
AZALEA-SAN SEBASTIAN
FTE.CARRANTONA-ENCOMIENDA PALACIOS
AV.AVIACION-Pº  EXTREMADURA
PZA.CORR.LICENCIADO A.MENA
DOMINGO PARRAGA-AV.REAL DE PINTO
AV.DOS DE MAYO-AV.FUENLABRADA
RIO TAJO-RIO VELEZ
VALLE DE IZALCO-AV.DE LAS LOMAS
ANTONIO MACHADO-GRANJA DEL CONDE
POZO DE LAS NIEVES-CIRCUNVALACION
Pº SAUCES Nº 8
CARPINTEROS-CENTRO MUNICIPAL DE EMPRESAS
BACARES-VELEZ RUBIO
AV.ENCUARTES-PANADEROS
AV.BARAJAS-PARQUE BOMBEROS
BATEL-PLAZA DEL MAR
AV.ESPAÑA-CENTRO COMERCIAL
VALDETORRES DE JARAMA-ROGELIO MUÑOZ
TOMAS REDONDO-VIA DE LOS POBLADOS
AREQUIPA-LOS EMIGRANTES
SAN FERNANDO-CAMPO DE FUTBOL
AV.ABRANTES-PAN BENDITO
CTRA.M507-PARQUE SAN SEBASTIAN
BACARES-VELEZ BLANCO
SENDA DEL INFANTE-GTA.PRADERA DE CORRALILLOS
AV.ENTREVIAS-RONDA SUR
JOAQUIN DICENTA-MARTIRES DE LA VENTILLA
AV.SAUCES

In [45]:
for station in df.itemLabel:
    print(station)

Estación de Bambú
Estación de Estadio Metropolitano
Estación de Las Tablas
Estación de Acacias / Embajadores
Estación de Ventura Rodríguez
Alameda de Osuna
Estación de Moncloa
Estación de Herrera Oria
Estación de Colombia
Plaza de Castilla
Estación de Barrio del Pilar
Concha Espina
Estación de Ibiza
Estación de Estrella
Estación de Feria de Madrid
Estación de Antón Martín
Estación de Barajas
Estación de Tirso de Molina
Estación de Aeropuerto T1-T2-T3
Estación de Menéndez Pelayo
Estación de Portazgo
Estación de Puente de Vallecas
República Argentina
Estación de Ríos Rosas
Estación de Canal
Estación de Alto del Arenal
Estación de Sierra de Guadalupe
Estación de Lista
Estación de Pacífico
Estación de Méndez Álvaro
Estación de Bilbao
Estación de San Cipriano
Estación de Vinateros
Estación de Tribunal
estación de Buenos Aires
Estación de Vicálvaro - Universidad Rey Juan Carlos
Estación de O'Donnell
Estación de Villaverde Alto
Estación de Quintana
Estación de Parque de las Avenidas
Estación 